In [ ]:
!pip install numpy==1.19.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM
import tensorflow.keras.layers as layers
from tensorflow.keras.layers.experimental.preprocessing import  StringLookup
import os
import numpy as np
import json
import pandas as pd

#Prepare data
from sklearn.model_selection import train_test_split
def load_image(path):

    img = tf.io.read_file("images/" + path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_image(img, channels=3)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [50, 200])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    img = img.numpy()
    return img

images = []
labels = []
num_labels = []
path_to_works = []
data = pd.read_csv("labels.csv")
with open("vocab.txt", 'r') as file:
    vocab = json.loads(file.read())

filenames = data["file_name"]

labels = data["text"]

characters = set(char for label in labels for char in label)
max_length = max([len(label) for label in labels])

#merge new and old dictionary
for c in characters:
    if c not in vocab:
        vocab.append(c)

with open("vocab.txt", 'w') as file:
    file.write(json.dumps(vocab))


#save data to several files  
for path in filenames[:20000]:
    images.append(load_image(path))
np.savez_compressed("x1.npz",a=images, b=np.array([]))

for path in filenames[20000:40000]:
    images.append(load_image(path))
np.savez_compressed("x2.npz",a=images, b=np.array([]))

for path in filenames[40000:60000]:
    images.append(load_image(path))
np.savez_compressed("x3.npz",a=images, b=np.array([]))

for path in filenames[60000:80000]:
    images.append(load_image(path))
np.savez_compressed("x4.npz",a=images, b=np.array([]))


for path in filenames[80000:100000]:
    images.append(load_image(path))
np.savez_compressed("x5.npz",a=images, b=np.array([]))

for path in filenames[100000:120000]:
    images.append(load_image(path))
np.savez_compressed("x6.npz",a=images, b=np.array([]))

for path in filenames[120000:140000]:
    images.append(load_image(path))
np.savez_compressed("x7.npz",a=images, b=np.array([]))

for path in filenames[160000:]:
    images.append(load_image(path))
np.savez_compressed("x8.npz",a=images, b=np.array([]))

#make all words be equal length
def proccess(lbl):

    if len(lbl) < max_length:
        for i in range(0, max_length - len(lbl)):
            lbl+=' '
    return lbl
labels = list(map(proccess, labels))

#tokenize
for label in labels:


    if label !=".DS_Store":
        nums = []

        for c in label.replace(".png",'').replace(".jpg",''):
            nums.append(vocab.index(c))



        num_labels.append(nums)
num_labels = np.array(num_labels)
np.save("labels.npy", num_labels)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
x1 = np.load("x1.npz")['a']

x2 = np.load("x2.npz")['a']

x3 = np.load("x3.npz")['a']

x4 = np.load("x4.npz")['a']

x5 = np.load("x5.npz")['a']

x6 = np.load("x6.npz")['a']

x7 = np.load("x7.npz")['a']

x8 = np.load("x8.npz")['a']



labels = np.load("labels.npy").astype("int64")


In [ ]:
import  tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.layers as layers

import tensorflow.keras
import os
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Bidirectional, LSTM, Reshape, Dropout
import numpy as np
import json

import numpy as np

class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = tf.keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred









img_input = Input(shape=(200, 50, 3), name="image_input",dtype="float32" )
lbl_input = Input(shape=(None,),dtype="float32")

x = layers.Conv2D(64,(3, 3), activation="relu",padding="same")(img_input)

x = layers.Conv2D(64,(3, 3), activation="relu",padding="same")(x)

x = layers.MaxPooling2D((2,2))(x)

x = layers.BatchNormalization()(x)




x = layers.Conv2D(128,(3, 3), activation="relu",padding="same")(x)
x = layers.Conv2D(128,(3, 3), activation="relu",padding="same")(x)



x = layers.MaxPooling2D((2,2))(x)


x = layers.BatchNormalization()(x)





x = layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(x)

x = layers.BatchNormalization()(x)

x = Reshape(((200 // 4), (50 // 4) * 64))(x)

x = Dense(64, activation="relu",kernel_initializer="he_normal")(x)
x = Dropout(0.4)(x)
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.35))(x)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.35), name="last_lstm")(x)

x = Dense(151, activation="softmax", name="target_dense")(x)
output = CTCLayer()(lbl_input, x)

model = Model([img_input, lbl_input], output)





class CERMetric(tf.keras.metrics.Metric):
    """
    A custom Keras metric to compute the Character Error Rate
    """
    def __init__(self, name='CER_metric', **kwargs):
        super(CERMetric, self).__init__(name=name, **kwargs)
        self.cer_accumulator = self.add_weight(name="total_cer", initializer="zeros")
        self.counter = self.add_weight(name="cer_count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        input_shape = K.shape(y_pred)
        input_length = tf.ones(shape=input_shape[0]) * K.cast(input_shape[1], 'float32')

        decode, log = K.ctc_decode(y_pred,
                                    input_length,
                                    greedy=True)

        decode = K.ctc_label_dense_to_sparse(decode[0], K.cast(input_length, 'int32'))
        y_true_sparse = K.ctc_label_dense_to_sparse(y_true, K.cast(input_length, 'int32'))

        decode = tf.sparse.retain(decode, tf.not_equal(decode.values, -1))
        distance = tf.edit_distance(decode, y_true_sparse, normalize=True)

        self.cer_accumulator.assign_add(tf.reduce_sum(distance))
        self.counter.assign_add(tf.cast(len(y_true),tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.cer_accumulator, self.counter)

    def reset_states(self):
        self.cer_accumulator.assign(0.0)
        self.counter.assign(0.0)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2,monitor="val_CER_metric",mode="min", restore_best_weights=True)
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(monitor='val_CER_metric', factor=0.2,patience=2,verbose=1, min_delta=0.006,mode="min", min_lr=0.0001)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("result.h5", monitor="val_CER_metric",mode='min', save_best_only=True, save_weights_only=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=[CERMetric()])

model.fit([x_train,y_train],y_train, epochs=30, batch_size=512, validation_split=0.05, callbacks=[early_stopping,model_checkpoint,reduce_lr])

In [ ]:
model.evaluate([x_test,y_test], y_test)